In [1]:
import os
os.environ["THEANO_FLAGS"] = "device=gpu"
from tasks import check

/home/mcherti/build/Theano/theano/tensor/signal/downsample.py:5: UserWarning: downsample module has been moved to the pool module.
  warnings.warn("downsample module has been moved to the pool module.")
/home/mcherti/build/Theano/theano/sandbox/cuda/nvcc_compiler.py:224: UserWarning: You have the cuda library directory in your lib_dirs. This has been known to cause problems and should not be done.
  warnings.warn("You have the cuda library directory in your "
Using gpu device 0: Tesla K20Xm (CNMeM is disabled, CuDNN 3007)
/home/mcherti/work/code/scikit-learn/sklearn/cross_validation.py:42: DeprecationWarning: This module has been deprecated in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.19.
  "This module will be removed in 0.19.", DeprecationWarning)


Couldn't import dot_parser, loading of dot files will not be possible.


In [2]:
v = check(what="notebook", 
          filename="training/39mnist/model.pkl", 
          dataset="digits")
          #force_w=64, force_h=64)
          #kw_load_data={"mode": "random"})

Loading data...


INFO:tasks:Loading data...


Loading the model...


INFO:tasks:Loading the model...


(None, 1, 28, 28)
Compiling the model...


INFO:tasks:Compiling the model...


In [ ]:
capsule, data, layers, w, h, c = v
print(w, h, c)
print(layers.keys())
#print(w, h, c)
#print(data.X.min(), data.X.max())

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
from lasagnekit.misc.plot_weights import tile_raster_images
import numpy as np
for name, layer in layers.items():
    #print(name, layer.output_shape)
    #if hasattr(layer, "W"):
    #    print(layer.W.get_value().shape)
    if name.startswith("out") and name != "output":
        W = layer.W.get_value()
        W = W.reshape((W.shape[0]*W.shape[1], W.shape[2], W.shape[3]))
        a, b = W.shape[1:]
        fig = plt.figure(figsize=(10, 10))
        s = int(np.sqrt(W.shape[0]))
        print(s, s, W.shape)
        plt.axis('off')
        img=tile_raster_images(W[:, ::-1, ::-1], (a, b), (s, s))
        plt.imshow(img, cmap="gray", interpolation='none')
        plt.show()

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
data.load()
X = capsule.preprocess(data.X)
X_rec = X
for i in range(1):
    X_rec = capsule.reconstruct(X_rec)

def p(X):
    shape = (X.shape[0], c, w, h)
    X = X.reshape(shape)
    X = X.transpose((0, 2, 3, 1))
    if c == 1:
        X = 1 - X[:, :, :, 0]
    else:
        pass
        #X = 1 - X
    return X

In [ ]:
if c == 1:
    cm = 'gray'
else:
    cm = None
for ind in range(15):
    plt.axis('off')
    plt.clf()
    plt.subplot(1, 2, 1)
    plt.axis('off')
    plt.imshow(p(X)[ind], interpolation='none', cmap=cm)
    plt.subplot(1, 2, 2)
    plt.imshow(p(X_rec)[ind], interpolation='none', cmap=cm)
    plt.axis('off')
    plt.show()

In [ ]:
import theano.tensor as T
import theano
from lasagne import layers as L
print(layers.keys())
x=T.tensor4()
o = [
    L.get_output(layers['out0'], x),
    L.get_output(layers['out1'], x),
    L.get_output(layers['out2'], x)
]
f = theano.function([x], o) 

o = [
    L.get_output(layers['wta_spatial_0'], x),
    L.get_output(layers['wta_spatial_1'], x),
    L.get_output(layers['wta_spatial_2'], x)
]
g = theano.function([x], o) 

l1, l2, l3 = T.tensor4(), T.tensor4(), T.tensor4()
o_l = [
    L.get_output(layers['out0'], {layers['wta_spatial_0']:l1}),
    L.get_output(layers['out1'], {layers['wta_spatial_1']:l2}),
    L.get_output(layers['out2'], {layers['wta_spatial_2']:l3})
]
g_forward = theano.function([l1, l2, l3], o_l)
g_rec = theano.function([l1, l2, l3],
                        L.get_output(layers["output"],
                        {layers['wta_spatial_0']:l1,
                         layers['wta_spatial_1']:l2,
                         layers['wta_spatial_2']:l3}))

c1 = L.get_output(layers['wta_spatial_0'], x)
c2 = L.get_output(layers['wta_spatial_1'], x)
c3 = L.get_output(layers['wta_spatial_2'], x)

loss1 = ((c1 - l1)**2).sum()
loss2 = ((c2 - l2)**2).sum()
loss3 = ((c3 - l3)**2).sum()

loss = loss1 + loss2 + loss3

get_loss = theano.function([x, l1, l2, l3], loss)
get_grad = theano.function([x, l1, l2, l3], theano.grad(loss, x))


recons_loss = ((L.get_output(layers["output"], x) - x)**2).sum()
get_recons_loss = theano.function([x], recons_loss)
get_recons_grad = theano.function([x], theano.grad(recons_loss, x))


#get_code_recons_loss = theano.function([x])

In [ ]:
X = capsule.preprocess(data.X)
a, b, c = g(X)
#print((a>0).sum(axis=(2, 3)).argmax(axis=1))
#print((b>0).sum(axis=(2, 3)).argmax(axis=1))

fig = plt.figure(figsize=(20, 20))
from lasagnekit.misc.plot_weights import grid_plot
grid_plot(b[0], imshow_options={"cmap": "gray"})
#img = tile_raster_images(a[3], (28, 28), (10, 10))

In [ ]:
X = capsule.preprocess(data.X)
a, b, c = g(X)
print(a.shape, b.shape, c.shape)
print((b>0).sum(axis=(2, 3)).max())
#a[:, 0:128] = 0
#b[:, 0:2] = 0
#c[:, 0:256] = 0
a[:, :] = np.random.uniform(size=a.shape).astype(np.float32)
b[:, :] = np.random.uniform(size=b.shape).astype(np.float32)
c[:, :] = np.random.uniform(size=c.shape).astype(np.float32)
#b[:, 10:20, 5:15, 5:15] = 800
a, b, c = g_forward(a, b, c)
def s(x):
    return 1/(1+np.exp(-x))
e = s(a + b + c)
print(a.shape)
img = tile_raster_images(e[0:30, 0], (64, 64), (10, 10))
fig = plt.figure(figsize=(10, 10))
plt.axis('off')
plt.imshow(img, cmap="gray")

In [ ]:
"""
from sklearn.decomposition import PCA
from sklearn.datasets import fetch_olivetti_faces
D  = fetch_olivetti_faces()
pca = PCA(n_components=100)
code = pca.fit_transform(D['data'])
data_rec = pca.inverse_transform(code)
data_rec = data_rec.reshape((data_rec.shape[0], 64, 64))
img = tile_raster_images(data_rec, (64, 64), (10, 10))
fig = plt.figure(figsize=(10, 10))
plt.axis('off')
plt.imshow(img, cmap="gray")
"""

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
data.load()
X = data.X
#X = np.random.uniform(size=data.X.shape).astype(np.float32)
X = capsule.preprocess(X)
X_rec = X

print(X_rec.shape)
img = tile_raster_images(X_rec[:, 0], (64, 64), (10, 10))
fig = plt.figure(figsize=(10, 10))
plt.axis('off')
plt.imshow(img, cmap="gray")

In [ ]:
#X_rec = np.zeros(X_rec.shape).astype(np.float32)
X_rec = X.copy()

In [ ]:
for i in range(10):
    a, b, c = g(X_rec)
    """
    for k in range(X_rec.shape[0]):
        A = np.random.randint(0, b.shape[2])
        B = A + 5
        C = np.random.randint(0, b.shape[2])
        D = C + 5
        idx = np.random.randint(b.shape[1])
        b[k, idx, A:B, C:D] *= 4
    """
    b*=4
    X_rec = g_rec(a, b, c)

In [ ]:
print(X_rec.shape)
score = ((X_rec - capsule.reconstruct(X_rec)) ** 2).sum(axis=(1, 2, 3))
ind = np.argsort(score)
X_rec = X_rec[ind]
img = tile_raster_images(X_rec[:, 0], (64, 64), (10, 10))
fig = plt.figure(figsize=(10, 10))
plt.axis('off')
plt.imshow(img, cmap="gray")

In [ ]:
theano.function()

In [ ]:

#X_dt = capsule.preprocess(data.X)
#X = np.random.uniform(size=X.shape).astype(np.float32)

X = np.random.uniform(size=X.shape).astype(np.float32)
#X = np.zeros(X.shape).astype(np.float32)
#a, b, c = g(X_dt)
#get_loss(X, a, b, c)

for i in range(1000):
    loss = get_recons_loss(X)
    print(loss)
    g = get_recons_grad(X)
    X -= 0.9 * g/(np.sqrt((g**2).sum()))
    if i % 10 == 0:
        score = ((X - capsule.reconstruct(X)) ** 2).sum(axis=(1, 2, 3))
        ind = np.argsort(score)
        X = X[ind]
        img = tile_raster_images(X[0:10, 0], (64, 64), (1, 10))
        fig = plt.figure(figsize=(10, 10))
        plt.axis('off')
        plt.imshow(img, cmap="gray")
        plt.show()